In [ ]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [ ]:
#Dataframe from 10stocks

filepath='file:////home/talentum/Project/Project_codes/10stocks_price.csv'
df = spark.read.option("header", True).csv(filepath)
df.show()

+------------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+
|      date_gsheets| AAPL|   KO| NFLX|BRK.B|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|
+------------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+
|  01/03/2012 16:00|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|
|  01/04/2012 16:00|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|
|  01/05/2012 16:00|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|
|  01/06/2012 16:00|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|
|  01/09/2012 16:00|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|
|  01/10/2012 16:00|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|
|  01/11/2012 16:00|60.36|34.03|13.16|   78| 38.7|182.32| 38.9| 59.4|18.88|28.23|
|  01/12/2012 16:00| 60.2|33.78|13.16| 78.5|38.73|180.55|38.92| 59.5|18.93|28.25|
|1/13/2012 16:00:00|59.97|33.49|13.48|77.77| 38.4|179.16|38.92|59.54|18.84|22.79|
|1/17/2012 16:00

In [ ]:
#string to date

from pyspark.sql.functions import *
df = df.withColumn("date_gsheets",to_date(df.date_gsheets,"MM/dd/yyyy HH:mm"))
df = df.withColumnRenamed('BRK.B', 'BRK')
df.show()

+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+
|date_gsheets| AAPL|   KO| NFLX|  BRK|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|
+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+
|  2012-01-03|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|
|  2012-01-04|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|
|  2012-01-05|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|
|  2012-01-06|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|
|  2012-01-09|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|
|  2012-01-10|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|
|  2012-01-11|60.36|34.03|13.16|   78| 38.7|182.32| 38.9| 59.4|18.88|28.23|
|  2012-01-12| 60.2|33.78|13.16| 78.5|38.73|180.55|38.92| 59.5|18.93|28.25|
|  2012-01-13|59.97|33.49|13.48|77.77| 38.4|179.16|38.92|59.54|18.84|22.79|
|  2012-01-17|60.67|33.67|13.53|77.97|38.48|   180|39.02|59.85|18.74| 26.6|
|  2012-01-1

In [ ]:
#Creating returns Dataframe

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df=df
my_window = Window.partitionBy().orderBy("date_gsheets")
for col_name in return_df.columns:
    if col_name!='date_gsheets':
     return_df = return_df.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df.columns:
    if c!='date_gsheets':
        return_df = return_df.withColumn(c, F.round(c, 6))

return_df=return_df.dropna()
return_df=return_df.drop('date_gsheets')
return_df.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|
| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292|
| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|
|-0.001492|      0.0| 0.137875|-0.001309|-0.004009|-0.005204| 0.001044| 0.003051|  0.01126| 0.012635|
| 0.003485| 0.005802|-0.024234| 0.014812|-0.003019|-0.001542| 0.005212|-0.002366|-0.007423| 0.013578|
|-0.001654| -0.01846|-0.038714| 0.007492|-0.023467| 0.005571| 0.008556| 0.006098| 0.008547| 0.022085|
|-0.002651|-0.007346|      0.0|  0.00641|  7.75E-4|-0.009708|  5.14E-4| 0.001684| 

In [ ]:
# calculating mean return for each stocks and creating a dtaframe
data1=return_df.columns
data2=[]
for c in return_df.columns:
    data2.append(return_df.select(mean(col(c))).collect()[0][0])
data = list(zip(data1, data2))
return_df_mean=spark.createDataFrame(data,["Stocks","Mean"])
return_df_mean = return_df_mean.withColumn("Mean", round(return_df_mean["Mean"], 6))
return_df_mean.show()

+------+--------+
|Stocks|    Mean|
+------+--------+
|  AAPL| 8.25E-4|
|    KO| 2.16E-4|
|  NFLX|0.002453|
|   BRK| 6.61E-4|
|   DIS|  7.5E-4|
|   IBM| -6.2E-5|
|    VZ| 2.42E-4|
|   WMT| 3.84E-4|
|    GE|  3.4E-5|
|  TSLA|0.002057|
+------+--------+



In [ ]:
# Extract a list of all the tickers in the dataframe
tickers = list(return_df.columns)
tickers

['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM', 'VZ', 'WMT', 'GE', 'TSLA']

In [ ]:
# Create 10 portfolios with each ensuing portfolio comprising of ONE additional stock.
portfolios = []

for i in range(1, len(tickers) + 1):
    portfolios.append(tickers[0:i])

In [ ]:
portfolios

[['AAPL'],
 ['AAPL', 'KO'],
 ['AAPL', 'KO', 'NFLX'],
 ['AAPL', 'KO', 'NFLX', 'BRK'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM', 'VZ'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM', 'VZ', 'WMT'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM', 'VZ', 'WMT', 'GE'],
 ['AAPL', 'KO', 'NFLX', 'BRK', 'DIS', 'IBM', 'VZ', 'WMT', 'GE', 'TSLA']]

In [ ]:
# Names to Portfolios
names = []

for i in range(1, len(portfolios)+1):
    names.append('Portfolio-'+str(i))


In [ ]:
names

['Portfolio-1',
 'Portfolio-2',
 'Portfolio-3',
 'Portfolio-4',
 'Portfolio-5',
 'Portfolio-6',
 'Portfolio-7',
 'Portfolio-8',
 'Portfolio-9',
 'Portfolio-10']

In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
from pyspark.mllib.linalg import DenseMatrix

"""
Returns the Annualised Expected Return of a portfolio.
Annualises the return using the 'crude' method.
"""
def getPortReturns():
    portfolio_returns = []
    for port,weight in zip(portfolios,portfolio_weightage):

        num_stocks=len(port)
        # creating weight matrix
        weight_matrix= DenseMatrix(1,num_stocks, weight)
        # Creating Row matrix for the Dense matrix to do dot_product using multiply function
        weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
        # Extracting mean return of the portfolio
        return_df_mean_list1=[]
        return_df_mean_list=[]
        for p in port:
            return_df_mean_list = return_df_mean.filter(return_df_mean.Stocks == p).select("Mean").rdd.flatMap(lambda x: x).collect()
            return_df_mean_list1.append(return_df_mean_list[0])

        return_df_matrix=DenseMatrix(len(return_df_mean_list1),1,return_df_mean_list1)
        # calculating dot_product of weight and mean return to get the expected return
        exp_ret_portfolio=weight_row_mat.multiply(return_df_matrix).rows.collect()
        # annual expected return through crude method
        exp_ret_portfolio_annual = exp_ret_portfolio[0][0] * 250
        # Returning the lsit of expected return of portfolios
        portfolio_returns.append(exp_ret_portfolio_annual)

    return portfolio_returns





In [ ]:
# Calculate the risk of each portfolio
from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
import numpy as np
# arguments are weight and return_df of portfolios #df_temp = return_df[port]
def getPortRisk(weights,df_temp):
    portfolio_risks = []

    # calculating the variance covariance matrix of return_df of portfolio

    return_rdd = df_temp.rdd.map(list)
    return_row_mat = RowMatrix(return_rdd)
    vcv_matrix=return_row_mat.computeCovariance()

    # Variance of the portfolio = weight matrix dot_product variance-covariance matricx dot_product weight transpose matrix
    weight_matrix= DenseMatrix(1, num_stocks, weights) # weight matrix
    weight_matrix_t= DenseMatrix(num_stocks, 1, weights) # weight matrix Transpose
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    vcv_w=weight_row_mat.multiply(vcv_matrix) # dot_product of weight matrix and vcv matrix
    var_portfolio = vcv_w.multiply(weight_matrix_t).rows.collect() # dot_product of (weight matrix and vcv matrix in prev line) with weight transpose

    # calculating standard deviation from variance and compute annual sd of the portfolio
    var_portfolio=var_portfolio[0][0]
    sd_portfolio=np.sqrt(var_portfolio)
    sd_portfolio_annual = sd_portfolio * np.sqrt(250)

    # returning the portfolio risk i.e. standard deviation computed annually
    portfolio_risks.append(sd_portfolio_annual)

    return portfolio_risks

In [ ]:
# Calculating the weightage of the stocks for the portfolio by minimising risks
from scipy.optimize import minimize
import numpy as np

portfolio_weightage=[]
for port in portfolios:

    num_stocks = len(return_df[port].columns)  # being the number of stocks (this is a 'global' variable)
    init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0) [Intially assume equal weightge]
    args=return_df[port] # second argument of getPortRisk() function
    bounds = tuple((0, 1) for i in range(num_stocks)) # weightage will be between 0 and 1
    cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1}) # sum of weightage will be 1
    # getting the optimised weightage through minimise() function with getPortRisk() as objective function and applying bounadries and constraints as adefined above
    results = minimize(fun=getPortRisk, x0=init_weights,args=args, bounds=bounds, constraints=cons)
    # converting the values to float and rounding off to 2 decimal places
    rounded_result = np.round(results.x, decimals=2)
    temp_rdd=sc.parallelize(rounded_result)
    temp_rdd=temp_rdd.map(lambda x:float(x))

    portfolio_weightage.append(temp_rdd.collect())


In [ ]:
portfolio_weightage

[[1.0],
 [0.18, 0.82],
 [0.17, 0.8, 0.03],
 [0.09, 0.49, 0.01, 0.4],
 [0.08, 0.47, 0.01, 0.33, 0.1],
 [0.07, 0.44, 0.01, 0.27, 0.09, 0.13],
 [0.06, 0.35, 0.01, 0.19, 0.06, 0.1, 0.21],
 [0.05, 0.3, 0.01, 0.16, 0.04, 0.09, 0.17, 0.18],
 [0.05, 0.3, 0.01, 0.14, 0.04, 0.09, 0.17, 0.18, 0.02],
 [0.05, 0.3, 0.01, 0.14, 0.04, 0.09, 0.17, 0.18, 0.02, 0.0]]

In [ ]:
# Calculating the investment amount of the portfolio i.e. latest closing price

latest_date = df.select(max("date_gsheets").alias("latest_date")).collect()[0]["latest_date"]
portfolio_amount = []

for port in portfolios:
    amount_rdd=df.filter(df.date_gsheets == latest_date).select(port).rdd.flatMap(lambda x: x).map(lambda x: float(x))
    total_amount= amount_rdd.collect()
    portfolio_amount.append(total_amount)



In [ ]:
portfolio_amount

[[169.23],
 [169.23, 45.88],
 [169.23, 45.88, 191.96],
 [169.23, 45.88, 191.96, 198.22],
 [169.23, 45.88, 191.96, 198.22, 107.51],
 [169.23, 45.88, 191.96, 198.22, 107.51, 153.42],
 [169.23, 45.88, 191.96, 198.22, 107.51, 153.42, 52.93],
 [169.23, 45.88, 191.96, 198.22, 107.51, 153.42, 52.93, 98.75],
 [169.23, 45.88, 191.96, 198.22, 107.51, 153.42, 52.93, 98.75, 17.45],
 [169.23, 45.88, 191.96, 198.22, 107.51, 153.42, 52.93, 98.75, 17.45, 311.35]]

In [ ]:
# function call for returns calulation as per thw portflio weightage
portfolio_return_wt=getPortReturns()


In [ ]:
portfolio_return_wt

[0.20625,
 0.08140499999999999,
 0.09666000000000001,
 0.117255,
 0.121295,
 0.10380750000000001,
 0.09120999999999999,
 0.092755,
 0.08962,
 0.08962]

In [ ]:
#Attaching SP500 details for calculating the stocks risk based on market risk i.e. beta value
# This step and next 3 steps can be skipped later while retieing from the database or the common file

filepath='file:////home/talentum/shared/Project/Project_codes/section5/googl_sp500_price.csv'
sp_500_df = spark.read.option("header", True).csv(filepath)
sp_500_df=sp_500_df.drop("price_googl")
sp_500_df.show()

+------------------+-----------+
|      date_gsheets|price_sp500|
+------------------+-----------+
|  01/03/2012 16:00|    1277.06|
|  01/04/2012 16:00|     1277.3|
|  01/05/2012 16:00|    1281.06|
|  01/06/2012 16:00|    1277.81|
|  01/09/2012 16:00|     1280.7|
|  01/10/2012 16:00|    1292.08|
|  01/11/2012 16:00|    1292.48|
|  01/12/2012 16:00|     1295.5|
|1/13/2012 16:00:00|    1289.09|
|1/17/2012 16:00:00|    1293.67|
|1/18/2012 16:00:00|    1308.04|
|1/19/2012 16:00:00|     1314.5|
|1/20/2012 16:00:00|    1315.38|
|1/23/2012 16:00:00|       1316|
|1/24/2012 16:00:00|    1314.63|
|1/25/2012 16:00:00|    1326.06|
|1/26/2012 16:00:00|    1318.45|
|1/27/2012 16:00:00|    1316.32|
|1/30/2012 16:00:00|    1313.02|
|1/31/2012 16:00:00|     1312.4|
+------------------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import *
sp_500_df = sp_500_df.withColumn("date_gsheets",to_date(sp_500_df.date_gsheets,"MM/dd/yyyy HH:mm"))
sp_500_df=sp_500_df.withColumnRenamed("price_sp500","SP500")
sp_500_df.show()

+------------+-------+
|date_gsheets|  SP500|
+------------+-------+
|  2012-01-03|1277.06|
|  2012-01-04| 1277.3|
|  2012-01-05|1281.06|
|  2012-01-06|1277.81|
|  2012-01-09| 1280.7|
|  2012-01-10|1292.08|
|  2012-01-11|1292.48|
|  2012-01-12| 1295.5|
|  2012-01-13|1289.09|
|  2012-01-17|1293.67|
|  2012-01-18|1308.04|
|  2012-01-19| 1314.5|
|  2012-01-20|1315.38|
|  2012-01-23|   1316|
|  2012-01-24|1314.63|
|  2012-01-25|1326.06|
|  2012-01-26|1318.45|
|  2012-01-27|1316.32|
|  2012-01-30|1313.02|
|  2012-01-31| 1312.4|
+------------+-------+
only showing top 20 rows



In [ ]:
#Creating returns Dataframe for SP500

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df_sp500=sp_500_df
my_window = Window.partitionBy().orderBy("date_gsheets")
for col_name in return_df_sp500.columns:
    if col_name!='date_gsheets':
        return_df_sp500 = return_df_sp500.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df_sp500.columns:
    if c!='date_gsheets':
        return_df_sp500 = return_df_sp500.withColumn(c, F.round(c, 6))

return_df_sp500=return_df_sp500.dropna()
return_df_sp500=return_df_sp500.drop('date_gsheets')
return_df_sp500.show()

+---------+
|    SP500|
+---------+
|  1.88E-4|
| 0.002944|
|-0.002537|
| 0.002262|
| 0.008886|
|   3.1E-4|
| 0.002337|
|-0.004948|
| 0.003553|
| 0.011108|
| 0.004939|
|  6.69E-4|
|  4.71E-4|
|-0.001041|
| 0.008694|
|-0.005739|
|-0.001616|
|-0.002507|
| -4.72E-4|
|   0.0089|
+---------+
only showing top 20 rows



In [ ]:
return_df_sp500 = return_df_sp500.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
return_df= return_df.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

return_df_sp500=return_df.join(return_df_sp500,"row_idx").drop("row_idx")
return_df=return_df.drop("row_idx")
return_df_sp500.show()


+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|    SP500|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|  1.88E-4|
| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292| 0.002944|
| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|-0.002537|
|-0.001492|      0.0| 0.137875|-0.001309|-0.004009|-0.005204| 0.001044| 0.003051|  0.01126| 0.012635| 0.002262|
| 0.003485| 0.005802|-0.024234| 0.014812|-0.003019|-0.001542| 0.005212|-0.002366|-0.007423| 0.013578| 0.008886|
|-0.001654| -0.01846|-0.038714| 0.007492|-0.023467| 0.005571| 0.008556| 0.006098| 0.008547| 0.022085|   

In [ ]:
# Calculating beta_value with Linear Regression

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=["SP500"], outputCol="features")
data_with_features = assembler.transform(return_df_sp500)
betaValues=[]
for col in return_df.columns:
# Create a LinearRegression model and fit it to the data
    lr = LinearRegression(featuresCol="features", labelCol=col) # passing each stock to LR as labelCol
    model = lr.fit(data_with_features)
    betaValues.append(model.coefficients[0]) # model.coefficients[0] gives beta value of specific stock

print(betaValues)


[0.3424976068226176, 0.28965309945713913, 0.382300280022112, 0.29603451961624316, 0.40271537864747203, 0.22921055347099153, 0.184579855470962, 0.12232567151346019, 0.3883340075100073, 0.5481792452397307]


In [ ]:
#Creating dataframe for Stocks with beta value
d1=return_df.columns
d2=[float(x) for x in betaValues]
d=zip(d1,d2)
beta_df=spark.createDataFrame(d,["Stocks","Beta"])
beta_df=beta_df.withColumn("Beta",round("Beta",2))
beta_df.show()

+------+----+
|Stocks|Beta|
+------+----+
|  AAPL|0.34|
|    KO|0.29|
|  NFLX|0.38|
|   BRK| 0.3|
|   DIS| 0.4|
|   IBM|0.23|
|    VZ|0.18|
|   WMT|0.12|
|    GE|0.39|
|  TSLA|0.55|
+------+----+



In [ ]:
# Grouping the beta values for the portfolio

beta_port=[]
for port in portfolios:
    beta_list=[]
    for p in port:
        beta_rdd=beta_df.filter(beta_df.Stocks == p).select("Beta").rdd.flatMap(lambda x: x).map(lambda x: float(x))
        beta_list.append(beta_rdd.collect())
        beta_list_rdd=sc.parallelize(beta_list)
        beta_list_f = beta_list_rdd.flatMap(lambda x: x).collect()
    beta_port.append(beta_list_f)

print(beta_port)

[[0.34], [0.34, 0.29], [0.34, 0.29, 0.38], [0.34, 0.29, 0.38, 0.3], [0.34, 0.29, 0.38, 0.3, 0.4], [0.34, 0.29, 0.38, 0.3, 0.4, 0.23], [0.34, 0.29, 0.38, 0.3, 0.4, 0.23, 0.18], [0.34, 0.29, 0.38, 0.3, 0.4, 0.23, 0.18, 0.12], [0.34, 0.29, 0.38, 0.3, 0.4, 0.23, 0.18, 0.12, 0.39], [0.34, 0.29, 0.38, 0.3, 0.4, 0.23, 0.18, 0.12, 0.39, 0.55]]


In [ ]:
# CAlculating the final Portfolio Dataframe incorporating Names, Portfolios, Weightage,return,Amount,Beta values

portfolio_return_wt=[float(x) for x in portfolio_return_wt]

data=list(zip(names,portfolios,portfolio_weightage,portfolio_return_wt,portfolio_amount,beta_port))
schema = StructType([
    StructField("Names", StringType(), True),
    StructField("Portfolios", ArrayType(StringType()), True),
    StructField("Weightage", ArrayType(FloatType()), True),
    StructField("Total_Returns", FloatType(), True),
    StructField("Amount", ArrayType(FloatType()), True),
    StructField("Beta_Port", ArrayType(FloatType()), True)])

final_portfolio_df = spark.createDataFrame(data, schema)

final_portfolio_df.show()

+------------+--------------------+--------------------+-------------+--------------------+--------------------+
|       Names|          Portfolios|           Weightage|Total_Returns|              Amount|           Beta_Port|
+------------+--------------------+--------------------+-------------+--------------------+--------------------+
| Portfolio-1|              [AAPL]|               [1.0]|      0.20625|            [169.23]|              [0.34]|
| Portfolio-2|          [AAPL, KO]|        [0.18, 0.82]|     0.081405|     [169.23, 45.88]|        [0.34, 0.29]|
| Portfolio-3|    [AAPL, KO, NFLX]|   [0.17, 0.8, 0.03]|      0.09666|[169.23, 45.88, 1...|  [0.34, 0.29, 0.38]|
| Portfolio-4|[AAPL, KO, NFLX, ...|[0.09, 0.49, 0.01...|     0.117255|[169.23, 45.88, 1...|[0.34, 0.29, 0.38...|
| Portfolio-5|[AAPL, KO, NFLX, ...|[0.08, 0.47, 0.01...|     0.121295|[169.23, 45.88, 1...|[0.34, 0.29, 0.38...|
| Portfolio-6|[AAPL, KO, NFLX, ...|[0.07, 0.44, 0.01...|    0.1038075|[169.23, 45.88, 1...|[0.34

In [ ]:
# CAlculating the Investment amount and beta value as per the  weightage

final_portfolio_df = final_portfolio_df.withColumn("Investment_Amount", expr("aggregate(zip_with(Weightage, Amount, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("Amount")

final_portfolio_df = final_portfolio_df.withColumn("Portfolio_Beta", expr("aggregate(zip_with(Weightage, Beta_Port, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("Beta_Port")

final_portfolio_df = final_portfolio_df.withColumn("Total_Returns", round(final_portfolio_df["Total_Returns"], 2)) \
.withColumn("Investment_Amount", round(final_portfolio_df["Investment_Amount"], 2)) \
.withColumn("Portfolio_Beta", round(final_portfolio_df["Portfolio_Beta"], 2))

final_portfolio_df.show()

+------------+--------------------+--------------------+-------------+-----------------+--------------+
|       Names|          Portfolios|           Weightage|Total_Returns|Investment_Amount|Portfolio_Beta|
+------------+--------------------+--------------------+-------------+-----------------+--------------+
| Portfolio-1|              [AAPL]|               [1.0]|         0.21|           169.23|          0.34|
| Portfolio-2|          [AAPL, KO]|        [0.18, 0.82]|         0.08|            68.08|           0.3|
| Portfolio-3|    [AAPL, KO, NFLX]|   [0.17, 0.8, 0.03]|          0.1|            71.23|           0.3|
| Portfolio-4|[AAPL, KO, NFLX, ...|[0.09, 0.49, 0.01...|         0.12|           118.92|           0.3|
| Portfolio-5|[AAPL, KO, NFLX, ...|[0.08, 0.47, 0.01...|         0.12|           113.19|          0.31|
| Portfolio-6|[AAPL, KO, NFLX, ...|[0.07, 0.44, 0.01...|          0.1|           117.09|           0.3|
| Portfolio-7|[AAPL, KO, NFLX, ...|[0.06, 0.35, 0.01...|        

In [ ]:
# Incorporate Volatility based on beta value , Beta VAlue > 1 High Volatile else Less Volatile

from pyspark.sql import functions as F

final_portfolio_df = final_portfolio_df.withColumn("Volatility", F.when(final_portfolio_df["Portfolio_Beta"] > 1, "High") \
                                                   .otherwise("Low"))
final_portfolio_df=final_portfolio_df.drop("Portfolio_Beta")
final_portfolio_df.show()




+------------+--------------------+--------------------+-------------+-----------------+----------+
|       Names|          Portfolios|           Weightage|Total_Returns|Investment_Amount|Volatility|
+------------+--------------------+--------------------+-------------+-----------------+----------+
| Portfolio-1|              [AAPL]|               [1.0]|         0.21|           169.23|       Low|
| Portfolio-2|          [AAPL, KO]|        [0.18, 0.82]|         0.08|            68.08|       Low|
| Portfolio-3|    [AAPL, KO, NFLX]|   [0.17, 0.8, 0.03]|          0.1|            71.23|       Low|
| Portfolio-4|[AAPL, KO, NFLX, ...|[0.09, 0.49, 0.01...|         0.12|           118.92|       Low|
| Portfolio-5|[AAPL, KO, NFLX, ...|[0.08, 0.47, 0.01...|         0.12|           113.19|       Low|
| Portfolio-6|[AAPL, KO, NFLX, ...|[0.07, 0.44, 0.01...|          0.1|           117.09|       Low|
| Portfolio-7|[AAPL, KO, NFLX, ...|[0.06, 0.35, 0.01...|         0.09|             98.7|       Low|
